In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

EPOCH_FILE = "measure-2025-11-27_15:31:22-epo-N.fif"
#EPOCH_FILE = "measure-2025-11-27_15:17:48-epo-H.fif"
epochs = mne.read_epochs(EPOCH_FILE, preload=True)

In [ ]:
# Apply bandpass filter 1 - 40 Hz
epochs.filter(l_freq=1, h_freq=40.0, fir_design='firwin')

In [ ]:
# Use max-min scaler to normalize data between -50 and 50

from sklearn.base import BaseEstimator, TransformerMixin

class MaxMinScaler(BaseEstimator, TransformerMixin):
    """Min-Max Scaler for normalizing data.
    Normalization occurs per channel.
    """

    def __init__(self, feature_range: tuple = (-1, 1), eps: float = 1e-8):
        """Min-Max Scaler for 3D arrays.

        Args:
            feature_range (tuple, optional): Desired range of transformed data. Defaults to (-1, 1).
            eps (float, optional): Small value to avoid division by zero. Defaults to 1e-8.
        """
        self.feature_range = feature_range
        self.eps = eps

    def fit(self, X: np.ndarray, y=None):
        # Calculate min and max values for each channel
        self._min = X.min(axis=(0, 2)).reshape((1, -1, 1))
        self._max = X.max(axis=(0, 2)).reshape((1, -1, 1))
        return self

    def transform(self, X: np.ndarray, y=None):
        denom = self._max - self._min
        denom = np.where(denom < self.eps, 1, denom)  # Prevent division by zero
        X = (X - self._min) / denom
        X = X * (self.feature_range[1] - self.feature_range[0]) + self.feature_range[0]
        return X

data = epochs.get_data(copy=False)
scaler = MaxMinScaler(feature_range=(-50, 50))
data = scaler.fit_transform(data)
epochs._data = data
print(f"Max value: {epochs.get_data(copy=True).max():.2e}")
print(f"Min value: {epochs.get_data(copy=True).min():.2e}")

Eye blink artifact seeing in raw data plot of epoch open eyes (filtered and normalized)
Typical for frontal channels. Occurs every 8 seconds (approx. 7 times per minute).

In [ ]:

print("Raw data plot of epoch open eyes (filtered and normalized):")
epochs[0].copy().crop(tmin=100, tmax=200).plot(scalings={"eeg":40, "grad":40, "mag":40})


In [ ]:
import mne
from mne.preprocessing import ICA

epoch_open_eyes = epochs[0].copy()
# Simplified workflow for manual component selection
ica = ICA(n_components=8, random_state=42, max_iter='auto')  # Use 7 for 8-channel system
ica.fit(epoch_open_eyes)

# Plot all components to identify eye blinks
ica.plot_components(inst=epoch_open_eyes)  # Shows topographies
ica.plot_sources(epoch_open_eyes)  # Shows time courses
# Eye blink components typically:
# - Have frontal topography (red/positive at front)
# - Show clear blink patterns in time course
# - Usually one of the first few components

# After identifying (e.g., component 0 is eye blinks):
ica.exclude = [2]  # Add all artifact component indices

# Apply ICA and update epochs._data directly (epochs[0] returns a copy, not a reference)
epoch_open_eyes_clean = ica.apply(epoch_open_eyes.copy())
epochs._data[0, :, :] = epoch_open_eyes_clean.get_data()[0, :, :]

epochs[0].crop(tmin=100, tmax=200).plot(scalings={"eeg":40, "grad":40, "mag":40})

Using second ica component to remove eye blink artifacts. It looks sufficient and good enough.

In [ ]:
# Topoplot of power in each frequency band for each epoch
# Frequency bands: theta (4-8 Hz), alpha (8-13 Hz), beta (13-30 Hz), gamma (30-40 Hz)

freq_bands = {
    'Theta (4-8 Hz)': (4, 8),
    'Alpha (8-13 Hz)': (8, 13),
    'Beta (13-30 Hz)': (13, 30),
    'Gamma (30-40 Hz)': (30, 40)
}

epoch_names = ['Open Eyes', 'Closed Eyes']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for epoch_idx in range(2):
    epoch = epochs[epoch_idx]
    
    # Compute PSD using Welch method
    spectrum = epoch.compute_psd(method='welch', fmin=1, fmax=40)
    
    for band_idx, (band_name, (fmin, fmax)) in enumerate(freq_bands.items()):
        ax = axes[epoch_idx, band_idx]
        
        # Plot topomap for this frequency band
        spectrum.plot_topomap(
            bands={band_name: (fmin, fmax)},
            axes=ax,
            show=False
        )
        
        if epoch_idx == 0:
            ax.set_title(band_name)
        if band_idx == 0:
            ax.set_ylabel(epoch_names[epoch_idx], fontsize=12)

plt.suptitle('Power Topomaps by Frequency Band', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
from mne.time_frequency import psd_array_welch


def compute_band_psd(epochs: mne.epochs, bands: dict) -> dict[str, np.ndarray]:
    X = epochs.get_data()
    sfreq = epochs.info['sfreq']
    n_epochs, n_channels, _ = X.shape

    # Compute PSD for each epoch and channel
    band_powers = {band: np.zeros((n_epochs, n_channels)) for band in bands.keys()}

    for ep_idx in range(n_epochs):
        for ch_idx in range(n_channels):
            # Compute PSD using Welch's method
            psds, freqs = psd_array_welch(
                X[ep_idx, ch_idx, :], 
                sfreq=sfreq, 
                fmin=0.5, 
                fmax=40.0,
                n_fft=int(sfreq * 2),  # 2-second windows
                n_overlap=int(sfreq)    # 1-second overlap
            )
            
            # Compute band power for each frequency band
            for band_name, (fmin, fmax) in bands.items():
                freq_mask = (freqs >= fmin) & (freqs <= fmax)
                band_powers[band_name][ep_idx, ch_idx] = np.mean(psds[freq_mask])
    return band_powers

In [ ]:
def plot_band_psd_comparison(psds: dict, epoch_names: list[str]):
    """Plot bar chart comparing relative average PSD across channels for each frequency band.
    
    Args:
        psds: Dictionary with band names as keys and (n_epochs, n_channels) arrays as values.
        epoch_names: List of epoch names for legend.
    """
    bands = list(psds.keys())
    n_bands = len(bands)
    
    # Average across channels
    avg_power = {band: psds[band].mean(axis=1) for band in bands}
    
    # Calculate total power for each epoch
    total_power = np.zeros(2)
    for band in bands:
        total_power += avg_power[band]
    
    # Calculate relative power (%)
    rel_power = {band: avg_power[band] / total_power * 100 for band in bands}
    
    x = np.arange(n_bands)
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    power_epoch0 = [rel_power[band][0] for band in bands]
    power_epoch1 = [rel_power[band][1] for band in bands]
    
    ax.bar(x - width/2, power_epoch0, width, label=epoch_names[0], color='steelblue')
    ax.bar(x + width/2, power_epoch1, width, label=epoch_names[1], color='coral')
    
    ax.set_xlabel('Frequency Band')
    ax.set_ylabel('Relative Power (%)')
    ax.set_title('Relative PSD Comparison: Open Eyes vs Closed Eyes (Channel Average)')
    ax.set_xticks(x)
    ax.set_xticklabels([b.capitalize() for b in bands])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

epoch_names = ['Open Eyes', 'Closed Eyes']
bands = {
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 40),
}
psds = compute_band_psd(epochs, bands)

plot_band_psd_comparison(psds, epoch_names)